<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI with a MCP Server

In [ ]:
# Install required packages

!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio

In [ ]:
# The MCP Server

from fastmcp import FastMCP
import nest_asyncio
import threading
import time

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer",
    instructions="This provides an up to date weather forecast for any location."
)

# Tool 1: Forecast
@mcp.tool("get_forecast")
def get_forecast(location: str):
    return {"forecast": f"Sunny in {location}"}

# Tool 2: Alerts
@mcp.tool(name="get_alerts")
def get_alerts(location: str):
    return {"alerts": f"No severe alerts currently for {location}"}

# Tool 3: Math
@mcp.tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

# Tool 4: Health Check
@mcp.tool
def health_check():
    """Returns the health status of the server."""
    return {"status": "ok"}

# Resource 1: Climate Data
@mcp.resource(uri="resource://climate")
def climate_data():
    """Return static climate information."""
    return {
        "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
        "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
        "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
    }

# Resource 2: Basic dynamic resource returning a string
@mcp.resource("resource://greeting")
def get_greeting() -> str:
    """Provides a simple greeting message."""
    return "Hello from FastMCP Resources!"

# Resource 3: Resource returning JSON data (dict is auto-serialized)
@mcp.resource("data://config")
def get_config() -> dict:
    """Provides application configuration as JSON."""
    return {
        "theme": "dark",
        "version": "1.2.0",
        "features": ["tools", "resources"],
    }

# Resource 4: Resource returning secret data
@mcp.resource("data://secret", enabled=False)
def get_secret_data():
    """This resource is currently disabled."""
    return "Secret data"

# Prompt 1: Analysis of Numerical data
@mcp.prompt
def analyze_data(data_points: list[float]) -> str:
    """Creates a prompt asking for analysis of numerical data."""
    formatted_data = ", ".join(str(point) for point in data_points)
    return f"Please analyze these data points: {formatted_data}"

# Define the function to run the server
def run_server():
  # Use transport="streamable-http" for compatibility with notebooks/Colab
  print("🚀 Starting FastMCP server in background thread...")
  mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

This code below sets up a test suite for the FastMCP server you started in the previous cell. It uses the fastmcp.Client to connect to the server and test its various functionalities: calling tools like get_forecast and multiply, reading resources such as climate_data, and retrieving prompts like analyze_data. The code then prints whether each test passed or failed, along with a summary of all tests. The output indicates that the client wasn't connected, which is a common issue when using fastmcp.Client outside of its recommended async with context manager.

{"forecast":"Sunny in London"}

CallToolResult(content=[TextContent(type='text', text='{"forecast":"Sunny in London"}', annotations=None, meta=None)], structured_content={'forecast': 'Sunny in London'}, meta=None, data={'forecast': 'Sunny in London'}, is_error=False)

Please analyze these data points: 10.5, 20.1, 30.0

meta=None description='Creates a prompt asking for analysis of numerical data.' messages=[PromptMessage(role='user', content=TextContent(type='text', text='Please analyze these data points: 10.5, 20.1, 30.0', annotations=None, meta=None))]


In [ ]:
# Create a MCP server test suite

import asyncio
from fastmcp import Client

# MCP_SERVER_URL is available from previous cells in the kernel state.
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def run_mcp_client_tests(server_url: str):
    print(f"🚀 Starting FastMCP client tests against: {server_url}")
    print("Ensure the MCP server is running in a separate thread/process.")

    all_results = []

    async with Client(server_url) as client:
        print("\n--- Inspecting Client Object ---")
        print(f"Client type: {type(client)}")
        print(f"Client dir: {dir(client)}")
        print("--------------------------------")

        # --- Test Tools ---
        print("\n--- Running Tool Tests ---")
        tool_test_cases = [
            {'name': 'get_forecast', 'arguments': {'location': 'London'}, 'expected_result': {"forecast": "Sunny in London"}},
            {'name': 'get_alerts', 'arguments': {'location': 'London'}, 'expected_result': {"alerts": "No severe alerts currently for London"}},
            {'name': 'multiply', 'arguments': {'a': 5, 'b': 3}, 'expected_result': 15.0},
            {'name': 'health_check', 'arguments': {}, 'expected_result': {"status": "ok"}}
        ]

        for test_case in tool_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected = test_case['expected_result']
            print(f"Testing tool: {name} with args: {args}")
            try:
                result = await client.call_tool(name, arguments=args)
                passed = (result.data == expected)
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Resources ---
        print("\n--- Running Resource Tests ---")
        resource_test_cases = [
            {'name': 'climate', 'expected_result': {
                "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
                "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
                "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
            }}
        ]

        for test_case in resource_test_cases:
            name = test_case['name']
            expected = test_case['expected_result']
            print(f"Testing resource: {name}")
            try:
                result = await client.read_resource(name)
                passed = (result == expected)
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Prompts ---
        print("\n--- Running Prompt Tests ---")
        prompt_test_cases = [
            {'name': 'analyze_data', 'arguments': {'data_points': [10.5, 20.1, 30.0]}, 'expected_prefix': "Please analyze these data points: 10.5, 20.1, 30.0"}
        ]

        for test_case in prompt_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected_prefix = test_case['expected_prefix']
            print(f"Testing prompt: {name} with args: {args}")
            try:
                result = await client.get_prompt(name, arguments=args)
                # Correctly access the text content of the prompt message
                message = result.messages[0]
                passed = (message.content.text == expected_prefix)
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected_prefix
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result='{message.content.text}'")
            except Exception as e:
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

    print("\n--- Test Summary ---")
    total_tests = len(all_results)
    passed_tests = sum(1 for r in all_results if r.get('passed', False))
    failed_tests = total_tests - passed_tests

    for res in all_results:
        status = 'PASSED' if res.get('passed', False) else 'FAILED'
        print(f"[{res['type'].upper()}] {res['name']}: {status}")
        if not res.get('passed', False):
            if 'error' in res:
                print(f"  Error: {res['error']}")
            else:
                print(f"  Expected: {res.get('expected')}")
                print(f"  Got: {res.get('result')}")
    print(f"\nTotal Tests: {total_tests}, Passed: {passed_tests}, Failed: {failed_tests}")

if __name__ == "__main__":
    # Make sure to run the MCP server cell (fYO0LykYQ05Y) before running this test suite.
    asyncio.run(run_mcp_client_tests(MCP_SERVER_URL))

In [ ]:
# Set your API key
client = OpenAI(api_key="your-api-key-here")

# Now you can use the OpenAI client
# The FastMCP server is running in the background on localhost:8000
# and can be accessed by your agent if configured to do so

# Example OpenAI Agent code
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "What's the weather like?"}
    ]
)

print(response.choices[0].message.content)